# Notebook: Chấm điểm IELTS Writing Task 2 bằng Qwen2-7B (Kaggle)



In [ ]:
# Section 1: Cài đặt và nhập các thư viện cần thiết
!pip install --upgrade pip
!pip install -q transformers accelerate torch jsonschema

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import base64
from jsonschema import validate, ValidationError

In [ ]:
# Section 2: Tải và nạp mô hình Qwen2-7B
MODEL_NAME = "Qwen/Qwen2-7B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model.eval()

In [ ]:
# Section 3: Định nghĩa schema đầu vào theo score_request.v1.json
score_request_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "score_request.v1.json",
    "title": "ScoreRequestV1",
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "task_type": {"type": "string", "enum": ["task1", "task2"]},
        "essay": {"type": "string", "minLength": 1, "maxLength": 20000},
        "question": {"type": "string", "minLength": 5, "maxLength": 1000},
        "image_base64": {"type": "string"},
        "options": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "max_evidence": {"type": "integer", "minimum": 1, "maximum": 3}
            }
        }
    },
    "required": ["task_type", "essay"],
    "allOf": [
        {
            "if": {"properties": {"task_type": {"const": "task1"}}},
            "then": {"required": ["essay"]}
        }
    ]
}

def validate_score_request(data):
    try:
        validate(instance=data, schema=score_request_schema)
        print("Input hợp lệ theo schema.")
        return True
    except ValidationError as e:
        print(f"Lỗi schema: {e}")
        return False

In [ ]:
# Section 4: Tiền xử lý dữ liệu đầu vào (essay, question, image_base64)
def preprocess_input(data):
    essay = data.get("essay", "")
    question = data.get("question", "")
    image = None
    if data.get("image_base64"):
        try:
            image = base64.b64decode(data["image_base64"])
        except Exception:
            image = None
    return essay, question, image

In [ ]:
# Section 5: Tạo hàm chấm điểm sử dụng Qwen2-7B

def get_standard_task2_prompt(essay, question):
    # Prompt chuẩn hóa giống src/app/prompts/task2.py
    return (
        f"You are an IELTS Writing Task 2 examiner.\n"
        f"Score the following essay according to the IELTS rubric (0-9) and provide feedback for each criterion (Task Response, Coherence and Cohesion, Lexical Resource, Grammatical Range and Accuracy).\n"
        f"Question: {question}\nEssay: {essay}\n"
        f"Return a JSON object with keys: overall, per_criterion (dict), feedback (dict)."
    )

def score_ielts_task2_qwen2(essay, question, model, tokenizer, max_new_tokens=512):
    prompt = get_standard_task2_prompt(essay, question)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    import re
    match = re.search(r"\{.*\}", result, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except Exception:
            return {"error": "Không parse được JSON từ kết quả model."}
    return {"error": "Không tìm thấy JSON trong kết quả model."}

In [ ]:
# Section 6: Chạy thử nghiệm chấm điểm với dữ liệu mẫu
sample_data = {
    "task_type": "task2",
    "essay": "In today’s world, technology has become an integral part of our lives. Some people believe that it has improved our quality of life, while others think it has made life more complicated. Discuss both views and give your own opinion.",
    "question": "Some people believe that technology has improved our quality of life, while others think it has made life more complicated. Discuss both views and give your own opinion.",
    "options": {"max_evidence": 2}
}

if validate_score_request(sample_data):
    essay, question, _ = preprocess_input(sample_data)
    result = score_ielts_task2_qwen2(essay, question, model, tokenizer)
else:
    result = {"error": "Input không hợp lệ."}

In [ ]:
# Section 7: Hiển thị kết quả chấm điểm
import pprint
pprint.pprint(result)

In [ ]:
# Section 5.1: Nạp prompt chuẩn từ mã nguồn (theo src/app/prompts/task2.py)
def get_standard_task2_prompt(essay, question):
    # Prompt chuẩn hóa giống src/app/prompts/task2.py
    return (
        f"You are an IELTS Writing Task 2 examiner.\n"
        f"Score the following essay according to the IELTS rubric (0-9) and provide feedback for each criterion (Task Response, Coherence and Cohesion, Lexical Resource, Grammatical Range and Accuracy).\n"
        f"Question: {question}\nEssay: {essay}\n"
        f"Return a JSON object with keys: overall, per_criterion (dict), feedback (dict)."
    )

# Sửa lại hàm chấm điểm để dùng prompt chuẩn

def score_ielts_task2_oss20b(essay, question, model, tokenizer, max_new_tokens=512):
    prompt = get_standard_task2_prompt(essay, question)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    import re
    match = re.search(r"\{.*\}", result, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except Exception:
            return {"error": "Không parse được JSON từ kết quả model."}
    return {"error": "Không tìm thấy JSON trong kết quả model."}

In [ ]:
# Section 8: Nạp và test batch với dataset mẫu (giả lập giống src/evaluation/datasets/hf_task2.py)
import pandas as pd

# Giả lập dataset mẫu (có thể thay bằng file thật nếu có)
dataset = [
    {
        "task_type": "task2",
        "essay": "Some people think that the best way to increase road safety is to increase the minimum legal age for driving cars or riding motorbikes. To what extent do you agree or disagree?",
        "question": "Some people think that the best way to increase road safety is to increase the minimum legal age for driving cars or riding motorbikes. To what extent do you agree or disagree?",
        "options": {"max_evidence": 2}
    },
    {
        "task_type": "task2",
        "essay": "Many people believe that social networking sites have a huge negative impact on both individuals and society. To what extent do you agree or disagree?",
        "question": "Many people believe that social networking sites have a huge negative impact on both individuals and society. To what extent do you agree or disagree?",
        "options": {"max_evidence": 2}
    }
]

results = []
for i, row in enumerate(dataset):
    if validate_score_request(row):
        essay, question, _ = preprocess_input(row)
        result = score_ielts_task2_qwen2(essay, question, model, tokenizer)
        results.append({"index": i, "result": result})
    else:
        results.append({"index": i, "result": {"error": "Input không hợp lệ."}})

# Hiển thị kết quả batch
df_results = pd.DataFrame(results)
df_results

In [ ]:
# Section 9: Đảm bảo output đúng format chuẩn score_response.v1.json
def validate_score_response_format(response):
    # Định nghĩa schema đơn giản hóa cho score_response.v1.json
    schema = {
        "type": "object",
        "properties": {
            "overall": {"type": "number"},
            "per_criterion": {"type": "object"},
            "feedback": {"type": "object"}
        },
        "required": ["overall", "per_criterion", "feedback"]
    }
    try:
        validate(instance=response, schema=schema)
        return True
    except ValidationError as e:
        print(f"Output không đúng format chuẩn: {e}")
        return False

# Kiểm tra kết quả batch
for r in results:
    print(f"Index {r['index']}:", validate_score_response_format(r['result']))

In [ ]:
# Section 10: Tải và chuẩn hóa dataset chillies/IELTS-writing-task-2-evaluation
from datasets import load_dataset
import pandas as pd

dataset_name = "chillies/IELTS-writing-task-2-evaluation"
df = load_dataset(dataset_name, split="test").to_pandas()
# Chuẩn hóa cột
if "band" in df.columns:
    df = df.rename(columns={"band": "band_true"})
if "id" not in df.columns:
    df["id"] = range(len(df))
df["word_count"] = df["essay"].astype(str).str.split().map(len)
df = df[["id", "prompt", "essay", "band_true", "word_count"]]
df.head()

In [ ]:
# Section 11: Batch scoring toàn bộ tập test bằng Qwen2-7B
from tqdm import tqdm

results = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    data = {
        "task_type": "task2",
        "essay": row["essay"],
        "question": row["prompt"],
        "options": {"max_evidence": 2}
    }
    if validate_score_request(data):
        essay, question, _ = preprocess_input(data)
        result = score_ielts_task2_qwen2(essay, question, model, tokenizer)
        band_pred = result.get("overall", None)
    else:
        band_pred = None
    results.append({"id": row["id"], "band_true": row["band_true"], "band_pred": band_pred})

df_pred = pd.DataFrame(results)
df_pred.head()

In [ ]:
# Section 12: Tính toán các chỉ số đánh giá MAE, RMSE, QWK, Pearson, Spearman
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import cohen_kappa_score
import numpy as np

def safe_round(x):
    try:
        return round(float(x))
    except:
        return np.nan

df_pred = df_pred.dropna(subset=["band_pred"]).copy()
df_pred["band_pred_round"] = df_pred["band_pred"].map(safe_round)

mae = mean_absolute_error(df_pred["band_true"], df_pred["band_pred"])
rmse = mean_squared_error(df_pred["band_true"], df_pred["band_pred"] , squared=False)
qwk = cohen_kappa_score(df_pred["band_true"], df_pred["band_pred_round"], weights="quadratic")
pearson = pearsonr(df_pred["band_true"], df_pred["band_pred"])[0]
spearman = spearmanr(df_pred["band_true"], df_pred["band_pred"])[0]

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"QWK: {qwk:.4f}")
print(f"Pearson: {pearson:.4f}")
print(f"Spearman: {spearman:.4f}")

In [ ]:
# Section 13: Hiển thị bảng so sánh kết quả dự đoán và thực tế
df_pred[["id", "band_true", "band_pred", "band_pred_round"]].head(20)